In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [4]:
!pip install tensorflow




In [9]:
#importing libraries
import sklearn
import numpy as np
from glob import glob
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline

import string
from os import listdir
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from pickle import dump
from string import punctuation

from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model


from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, LSTM, MaxPooling1D, concatenate


from pickle import load

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

import csv, string

In [10]:
categories = ['Gender abusive hate', 'Geopoitical Hate','Religious Hate','Political Normal','Political Hate','Personal Hate']
num_of_labels=len(categories)
categories_dict={key:value for value,key in enumerate(categories)}
categories_inverse_dict={key:value for key,value in enumerate(categories)}
print(categories_dict)

{'Gender abusive hate': 0, 'Geopoitical Hate': 1, 'Religious Hate': 2, 'Political Normal': 3, 'Political Hate': 4, 'Personal Hate': 5}


In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
stop_words='/content/drive/MyDrive/Thesis/stopwords-bn.txt'
text_data=[]
with open(stop_words,'r',encoding='utf-8') as temp_output_file:
    reader=csv.reader(temp_output_file, delimiter='\n')
    for row in reader:
        text_data.append(row)
stop_word_list=[x[0] for x in text_data]
print(stop_word_list[0:2])
print(type(stop_word_list))

['অতএব', 'অথচ']
<class 'list'>


# New Section

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
def save_dataset(dataset, filename):
    dump(dataset, open(filename,'wb'))
    print('Saved :%s' % filename)

def load_doc(filename):
    file=open(filename,'r',encoding='utf-8')
    text=file.read()
    file.close()
    return text

def clean_doc(doc, stop_word_list):
    # split into tokens by white space
    sentences=list()

    for sentence in doc:
        # remove punctuation from each token

        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in sentence.split(' ')]  # I belive word in sentence
        sentences.append(' '.join(tokens))
    return sentences

In [24]:
import pandas as pd

file_path = "/content/drive/MyDrive/Thesis/bengali_hate_v2.0.csv"

df = pd.read_csv(file_path, encoding="utf-8")  # Load the dataset
print(df.head())  # Print first few rows to check structure
print(df.columns)  # Show column names


                                                text         label  target
0  বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না - পোল...      Personal       0
1  এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে ...     Political       1
2  পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...  Geopolitical       3
3  ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভা...  Geopolitical       3
4            খানকির পুলা মালায়নদের মেরে সাফা করে ফেল      Personal       0
Index(['text', 'label', 'target'], dtype='object')


In [27]:
import pandas as pd

def load_data_and_labels(filename, stop_word_list):
    df = pd.read_csv(filename, encoding="utf-8")  # Load CSV properly

    x_doc = df["text"].tolist()  # Extract text
    label_names = df["label"].tolist()  # Extract labels

    categories_dict = {
        "Personal": 0,
        "Political": 1,
        "Religious":2,
        "Geopolitical": 3

    }  # Define your category mapping

    labels = []
    for label_name in label_names:
        if label_name not in categories_dict:
            print(f"Warning: Label '{label_name}' not found in categories_dict")
            continue  # Skip unknown labels
        labels.append(categories_dict[label_name])

    trainX = clean_doc(x_doc, stop_word_list)  # Clean text
    return trainX, labels

X_train, y_train = load_data_and_labels('/content/drive/MyDrive/Thesis/bengali_hate_v2.0.csv', stop_word_list)


In [28]:
import pandas as pd

def load_data_and_labels(filename, stop_word_list):
    df = pd.read_csv(filename, encoding="utf-8")  # Load CSV properly
    print(f"Dataset loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")  # Debugging

    x_doc = df["text"].tolist()  # Extract text
    label_names = df["label"].tolist()  # Extract labels

    categories_dict = {
        "Personal": 0,
        "Political": 1,
        "Religious": 2,
        "Geopolitical": 3
    }  # Define your category mapping

    labels = []
    unknown_labels = 0

    for label_name in label_names:
        if label_name not in categories_dict:
            unknown_labels += 1
            continue  # Skip unknown labels
        labels.append(categories_dict[label_name])

    print(f"Processed {len(labels)} labels. Skipped {unknown_labels} unknown labels.")  # Debugging

    trainX = clean_doc(x_doc, stop_word_list)  # Clean text
    print(f"Preprocessed text data: {len(trainX)} samples.")  # Debugging

    return trainX, labels

X_train, y_train = load_data_and_labels('/content/drive/MyDrive/Thesis/bengali_hate_v2.0.csv', stop_word_list)

print(f"X_train sample: {X_train[:5]}")  # Show first 5 processed texts
print(f"y_train sample: {y_train[:5]}")  # Show first 5 labels


Dataset loaded successfully with 5698 rows and 3 columns.
Processed 5698 labels. Skipped 0 unknown labels.
Preprocessed text data: 5698 samples.
X_train sample: ['বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না  পোলাপান এর চোখ কিভাবে ঠিক থাকবে', 'এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে বছরের পর বছর তাদের আরো এমন রাস্তায় রাস্তায় কাঁদতে হবে ', 'পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস্থা ধ্বংসের জন্য ভারতের প্রত্যক্ষ সহযোগিতায় এই হত্যাকা ঘটানো হয়েছিল ', 'ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভারতের অর্থনীতি নিয়ে ভারত সরকার আছে। আদার বেপারী জাহাজের খোঁজ নিয়ে লাভ নাই।', 'খানকির পুলা মালায়নদের মেরে সাফা করে ফেল']
y_train sample: [0, 1, 3, 3, 0]


In [30]:
def load_data_and_labels(filename, stop_word_list):
    doc = load_doc(filename)
    x_doc = []
    label = []

    first_line = True  # Flag to skip header

    for line in doc.split('\n'):
        if first_line:  # Skip the first line (header)
            first_line = False
            continue

        x_and_label = line.split(',')

        # Ensure there are at least two elements (text + label)
        if len(x_and_label) < 2:
            continue  # Skip malformed lines

        text, label_name = x_and_label[1], x_and_label[0]

        if label_name not in categories_dict:
            #print(f"Warning: Label '{label_name}' not found in categories_dict")  # Debugging
            continue  # Skip unknown labels

        x_doc.append(text)
        label.append(categories_dict[label_name])  # Convert label to numeric

    trainX = clean_doc(x_doc, stop_word_list)
    return trainX, label


In [31]:
X_train, y_train = load_data_and_labels('/content/drive/MyDrive/Thesis/bengali_hate_v2.0.csv',stop_word_list)

In [34]:
def load_data_and_labels(filename, stop_word_list):
    df = pd.read_csv(filename, encoding="utf-8")  # Load CSV properly

    x_doc = df["text"].tolist()  # Extract text
    label_names = df["label"].tolist()  # Extract labels

    categories_dict = {
        "Personal": 0,
        "Political": 1,
        "Religious": 2,
        "Geopolitical": 3
    }  # Define your category mapping

    labels = []
    for label_name in label_names:
        if label_name not in categories_dict:
            print(f"Warning: Label '{label_name}' not found in categories_dict")
            continue  # Skip unknown labels
        labels.append(categories_dict[label_name])

    print(f"Total samples before cleaning: {len(x_doc)}")  # Check if data is loaded
    print("First 10 raw texts:", x_doc[:10])  # See raw data before cleaning

    trainX = clean_doc(x_doc, stop_word_list)  # Clean text

    print(f"Total samples after cleaning: {len(trainX)}")  # Check if cleaning is removing everything
    print("First 10 cleaned texts:", trainX[:10])  # See processed data

    return trainX, labels

X_train, y_train = load_data_and_labels('/content/drive/MyDrive/Thesis/bengali_hate_v2.0.csv', stop_word_list)


Total samples before cleaning: 5698
First 10 raw texts: ['বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না - পোলাপান এর চোখ কিভাবে ঠিক থাকবে!', 'এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে বছরের পর বছর, তাদের আরো এমন রাস্তায় রাস্তায় কাঁদতে হবে ', 'পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস্থা ধ্বংসের জন্য ভারতের প্রত্যক্ষ সহযোগিতায় এই হত্যাকা- ঘটানো হয়েছিল ', 'ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভারতের অর্থনীতি নিয়ে ভারত সরকার আছে। আদার বেপারী জাহাজের খোঁজ নিয়ে লাভ নাই।', 'খানকির পুলা মালায়নদের মেরে সাফা করে ফেল', 'শুয়োর এর বাচ্চা দুরগারে চুদি কুত্তা দিয়া কালিরে চুদি পাঠা দিয়া তোর মারে চুদি আমার সোনা দিয়া সিং', 'মাগীর পাছায় লাথি মেরে ওই প্লাটফর্মে রেখে আসা উচিত ', 'আমরা তো দেখলাম ইউটিউব এর মাধ্যমে গাড়ি আওয়ামী লীগ এর লোকজনে পোড়ালো। অথচ মামলা করে দিলো বিএনপির নামে।', 'সানিউর কুত্তার বাচ্চা, তুর বোনেরে চুদমু তুর কান্দে ফালাইয়া, তুর মাইরে চুদমো তুর বাবার কান্দে ফালাইয়া, বান্দির বাচ্চা রেন্ডির সন্তান,', 'ভারতে মুসলমান খেদাও আন্দোলনের নামে প্রচার করা হচ্ছে মুসলিম বিদ্বেষী পো

In [35]:
def create_tokenizer(lines):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def encode_text(tokenizer, lines, length):
    #print(lines)
    encoded=tokenizer.texts_to_sequences(lines)
    print(encoded[0:10])
    padded= pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [36]:
train_tokenizer=create_tokenizer(X_train)

In [37]:
trainLength=300
vocab_size=len(train_tokenizer.word_index)+1
print('Max document length: %d' % trainLength)
print('Vocabulary size: %d' % vocab_size)
print()
X_train = encode_text(train_tokenizer, X_train, trainLength)
print(X_train.shape)
#trainX[2]

Max document length: 300
Vocabulary size: 19310

[[4369, 352, 3218, 11, 36, 290, 795, 107, 338, 1, 2584, 24, 795, 271, 107, 298], [4, 84, 9, 105, 4370, 44, 2585, 358, 669, 96, 376, 28, 253, 114, 415, 415, 3219, 18], [2175, 3220, 33, 1875, 1209, 2586, 14, 8, 2587, 4371, 4, 6995, 4372, 907], [8, 2588, 281, 259, 2176, 18, 60, 8, 2588, 281, 6, 84, 328, 6996, 6997, 6998, 4373, 281, 299, 239], [78, 519, 4374, 144, 6999, 2, 1342], [1668, 24, 62, 7000, 344, 267, 1102, 4375, 344, 1669, 1102, 31, 307, 344, 36, 796, 1102, 2177], [606, 4376, 429, 144, 240, 4377, 743, 744, 123], [48, 11, 854, 1876, 24, 520, 3221, 145, 151, 24, 7001, 7002, 329, 1103, 2, 1670, 1025, 7003], [7004, 215, 62, 544, 4378, 4379, 544, 4380, 4381, 544, 1492, 7005, 544, 640, 4380, 4381, 2589, 62, 7006, 447], [47, 143, 7007, 1877, 254, 607, 30, 102, 40, 1878, 7008]]
(5698, 300)


In [38]:
word_vectors = Word2Vec.load('C:/Users/admin-karim/Desktop/BengWord2Vec/posts.bin')

NotImplementedError: Unable to handle scheme 'c', expected one of ('', 'file', 'ftp', 'ftps', 'gs', 'hdfs', 'http', 'https', 'viewfs', 'webhdfs'). Extra dependencies required by 'c' may be missing. See <https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst> for details.

In [ ]:
NUM_WORDS=20000
EMBEDDING_DIM=300

vocabulary_size=len(train_tokenizer.word_index)+1
word_index=tokenizer.word_index
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector=word_vectors[word]
        embedding_matrix[i]=embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

c:\users\admin-karim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [ ]:
embedding_layer=Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)

In [ ]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	input1 = Input(shape=(length,))
	embedding_layer_1 = embedding_layer(input1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding_layer_1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)

	# channel 2
	input2 = Input(shape=(length,))
	embedding_layer_2 = embedding_layer(input2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding_layer_2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)

	# channel 3
	input3 = Input(shape=(length,))
	embedding_layer_3 = embedding_layer(input3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding_layer_3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)

	# merge
	CNN_layer = concatenate([flat1, flat2, flat3])

	# LSTM
	x = embedding_layer(input3)
	LSTM_layer = LSTM(128)(x)

	CNN_LSTM_layer = concatenate([LSTM_layer, CNN_layer])

	# interpretation
	dense1 = Dense(10, activation='relu')(CNN_LSTM_layer)
	outputs = Dense(num_of_labels, activation='softmax')(dense1)
	model = Model(inputs=[input1, input2, input3], outputs=outputs)

	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

	# summarize
	print(model.summary())

	return model

In [ ]:
model= define_model(EMBEDDING_DIM,vocabulary_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 300, 300)     1025100     input_4[0][0]                    
                                                                 input_5[0][0]                    
          

In [ ]:
from keras.utils import to_categorical

def toCategorical(y):
    y = to_categorical(y)
    return y

In [ ]:
y_train = toCategorical(y_train)

In [ ]:
model.fit([X_train,X_train,X_train], y_train, epochs=20, batch_size=128, validation_split=0.2)

Train on 701 samples, validate on 176 samples
Epoch 1/20
701/701 [==============================] - 12s 17ms/step - loss: 1.6760 - acc: 0.2539 - val_loss: 3.3705 - val_acc: 0.0398
Epoch 2/20
701/701 [==============================] - 9s 14ms/step - loss: 1.3261 - acc: 0.4579 - val_loss: 5.2184 - val_acc: 0.0000e+00
Epoch 3/20
701/701 [==============================] - 9s 13ms/step - loss: 1.1300 - acc: 0.5449 - val_loss: 6.8787 - val_acc: 0.0000e+00
Epoch 4/20
701/701 [==============================] - 12s 16ms/step - loss: 1.0058 - acc: 0.5877 - val_loss: 8.1012 - val_acc: 0.0000e+00
Epoch 5/20
701/701 [==============================] - 11s 16ms/step - loss: 0.8959 - acc: 0.6391 - val_loss: 9.2797 - val_acc: 0.0000e+00
Epoch 6/20
701/701 [==============================] - 22s 32ms/step - loss: 0.8244 - acc: 0.6534 - val_loss: 9.5983 - val_acc: 0.0000e+00
Epoch 7/20
701/701 [==============================] - 24s 34ms/step - loss: 0.7083 - acc: 0.7361 - val_loss: 9.9701 - val_acc: 0.005

In [ ]:
# save the model
model.save('MConv_LSTM_model.h5')

In [ ]:
X_test, y_test = load_data_and_labels('Hate_Speech_Test.csv',stop_word_list)
y_test = toCategorical(y_test)

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

tokenizer_test = create_tokenizer(X_test)

trainLength=300
vocab_size=len(tokenizer_test.word_index)+1
print('Max document length: %d' % trainLength)
print('Vocabulary size: %d' % vocab_size)
print()
X_test = encode_text(tokenizer_test, X_test, trainLength)
print(X_test.shape)
#trainX[2]

# load the model
model = load_model('MConv_LSTM_model.h5')

# evaluate model on test dataset dataset
loss, acc = model.evaluate([X_test,X_test,X_test], y_test, verbose=0)
print('Test accuracy: %f' % (acc*100))

Max document length: 300
Vocabulary size: 1616

[[222, 409, 410, 411, 412, 22], [145, 47, 81, 48, 146, 413, 414, 82, 109], [415, 223, 42, 224, 5, 416], [417, 418, 419, 225, 23], [110, 10, 420, 226, 227, 421, 228, 422, 423, 424, 229, 230, 147, 231, 1, 36, 425, 37, 426], [148, 427, 38, 428, 232, 83, 111, 38, 429, 1], [26, 233, 430, 149], [431, 432, 14, 433, 434], [435, 23, 234, 235, 236, 436], [26, 17, 47]]
(323, 300)
Test accuracy: 30.650155
